# Project Details

### AIM : 
<b> To implement a language model which will perform better in playing the Hangman game as compared to the traditional models which depend on unigram,bigram and n-gram models</b>

### Creating test beds
##### Corpus Description
<ul>
<li>Name: <b> corncob_lowercase.txt.</b></li>
<li>Description: <b> Contains 58110 lowercase english words</b></li>
</ul>

##### <b>Stratergy :</b> The model will evaluated using the concept of 10-cross validation
<small>In k-fold cross-validation, the original sample is randomly partitioned into k equal size subsamples. Of the k subsamples, a single subsample is retained as the validation data for testing the model, and the remaining k-1 subsamples are used as training data. The cross-validation process is then repeated k times (the folds), with each of the k subsamples used exactly once as the validation data. The k results from the folds can then be averaged (or otherwise combined) to produce a single estimation. The advantage of this method is that all observations are used for both training and validation, and each observation is used for validation exactly once.</small>

In [ ]:
import random
import operator
words = []
with open("train.txt", mode="r") as myFile:
    for line in myFile:
        try:
            words.append(line.strip().encode("utf-8"))
        except:
            pass

#### Dividing the Array into 10 equal parts

<p>Since we would be doing 10-fold validation, we need to divide the shuffled list of words into 10 roughly equal parts. We would use 9 of them as train and 1 as test set.</p>

In [ ]:
sub_parts = {}
for i in xrange(len(words)):
    key = i%10
    if key in sub_parts:
        sub_parts[key].append(words[i])
    else:
        sub_parts[key] = [words[i]]        

#### Calculating alphabet probabilities in the list of words

<p>This function returns the probability s of a particular alphabet, calculated on the basis of its occurence in the word list passed it to it.</p>

In [ ]:
def count_alpha_occurence(list_of_words):
    freq_dict = {}
    total = 0
    for word in list_of_words:
        for char in word:
            total += 1
            if char in freq_dict:
                freq_dict[char] = freq_dict[char] + 1
            else:
                freq_dict[char] = 1
    for key in freq_dict:
        freq_dict[key] = float(freq_dict[key])/total
    
    # sorting the dictionary based on values and reversing it to have the list of alphabets in decreasing order of their usage
    freq_dict =  sorted(freq_dict.items(), key=operator.itemgetter(1),reverse=True)
    return freq_dict

#### Calculating the conditional probabilties of an alphabet

<p> For a given a alphabet 'a' it returns a dictionary where key is 'b' and value is the probability of b occuring in a word given that 'a' is present in the word. This is to exploit the fact that occurence of an alphabet depends on the company it keeps.</p>

### Personal Details

<ol>
    <li>Name: <b>G V Sandeep</b></li>
    <li>College: <b>BITS - Pilani, Hyderabad Campus</b></li>
    <li>Github: <a href="https://github.com/greetsandeep/">greetsandeep</a></li>
</ol>

This code is open sourced and can be found at : <a href"https://github.com/greetsandeep/ACM_SummerSchool/tree/master/Improvising%20Hangman">Improvising Hangman</a>